### Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig
import glob 
import os
import fasttext
import re
from sklearn import metrics
# import pyLDAvis.gensim
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.semi_supervised import LabelSpreading
from sklearn.preprocessing import FunctionTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import FunctionTransformer
from sklearn import preprocessing
from sklearn import metrics
#from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression

import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
import numpy as np
import torch
import dill
import pickle


#### Setting up device for GPU usage

In [2]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [3]:
#lang = 'english'
#STOPWORDS = stopwords.words(LANG)

#### Setting display options

In [2]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 500)
pd.set_option('display.max_colwidth', 0)

### Import Train, Val & Test Sets

In [3]:
train_data = pd.read_csv('./data/Train.csv')
val_data = pd.read_csv('./data/Valid.csv')
test_data = pd.read_csv('./data/Test.csv')

In [4]:
train_data.label.value_counts()

0    20019
1    19981
Name: label, dtype: int64

In [5]:
val_data.label.value_counts()

1    2514
0    2486
Name: label, dtype: int64

In [6]:
test_data.label.value_counts()

1    2505
0    2495
Name: label, dtype: int64

In [7]:
train_data.sample(10)

,text,label
38431,"This film is to the F.B.I.'s history as Knott's Berry Farm is to the old west. Shamelessly sanitized version of the Federal Bureau of Investigation fight against crime. Hoover's heavy hand (did he have any other kind?) shows throughout with teevee quality script-reading actors, cheesy sets, cheap sound effects and lighting 101. With Jimmy Stewart at 20% of dramatic capacity, Vera Miles chewing the scenery, the film features every c-lister known in the mid-fifties with nary a hint of irony or humor, from the 'Amazon jungle' to the 'back yard barbecue', everything reeks of sound stages and back lots. Even the gunshots are canned and familiar. I imagine Mervyn Leroy got drunk every night. Except for a few (very few) interesting exterior establishing shots, nothing here of note beyond a curio.",0
37703,"I'm sorry, but this is such a bad movie it's hilarious. Football hooligans arguing in a travel lodge? Suits? Shades?! Alan clearly had no idea what he was talking about when he made this, it is as far from the truth as you can get.<br /><br />The casting was atrocious...Gary Oldman as a football hooligan? He doesn't look scary, act scary or even come across as someone who would like football. And as for Yeti? What the hell? Suits, shades and sitting in a travelodge childishly taunting each other with ""its about time you got your nappy on"". Please.<br /><br />And the Yeti's gang spraying the ICC's underwear? <br /><br />I don't see how anyone can even take this film seriously!.<br /><br />4/10. Its possibly the most inaccurate portrayal of the crisis of the late eighties hooliganism i have ever seen.",0
23487,"Margaret Mitchell spins in her grave every time somebody watches this mess! Fine costuming and sets can't even begin to overwhelm lackluster performances by Joanne Whalley (as the title character) and the ever-bland Timothy Dalton (as Rhett). Even worse than the acting--and perhaps partially explaining it--is the script, which is astoundingly cliched and predictable. Add to that hellishly bad script a score that'll have you cringing, and you've got a disaster I wouldn't wish on any viewer. SCARLETT is just amazingly lousy, and I can't imagine how it ever got made, much less made it to video.",0
6253,"Just recently, I've been obsessing over and anticipating this movie so much that I almost had to see it. Well, having just seen it today, the 5.8 rating is completely understandable. I think that if you anticipate something so much that it becomes a dire need, it turns out not to be worth it. <br /><br />Sure, The Hills Have Eyes 2 has its moments. It has a very cool and well-developed storyline that ties in well with the actual product itself, but the whole thing is so self-indulged that it becomes so hard to follow. And if it weren't for Wes Craven's production on this film, it wouldn't be anything to do with The original remake. <br /><br />But the whole thing makes you go ""Is this supposed to be horror or COMEDY?"" because there are lots of ridiculous, randomly placed jump moments and stupid one liners (I.E. ""There's a hand in the sh**er!"" or ""You motherfu**er! I'll kill you all damn sons of b**tches!"") and the acting (God don't even remind me how bad it was.<br /><br />STORYLINE: (this part contains spoilers, beware!) The movie begins with a woman giving birth to a mutant baby (ooh la la!), and then the screen fades to black with the movie's title appearing, and a monologue. Then we go to this office where there are randomly placed war veteran mannequins. We find that this is for this one scientist keeping track of people looking for mutants. The box to keep track of audio feeds is gone, and everyone dies! After that tone-setting opening, you'd expect more.<br /><br />Then, we go to this one team of military recruits training in Baghdad. As the captain parades them ""A good job at stupidity"", their last day of training is in New Mexico, the desert where the family in the last THHE had stayed because

###### Let's take a look at some statistical figures for word count. 

In [8]:
train_data['text']= train_data['text'].astype(str)
train_data['word_count'] = train_data['text'].apply(lambda x : len(x.split()))

In [9]:
train_data.word_count.describe()

count    40000.000000
mean     231.339250  
std      171.194123  
min      4.000000    
25%      126.000000  
50%      173.000000  
75%      282.000000  
max      2470.000000 
Name: word_count, dtype: float64

### Train SVC model 

In [10]:
TF_PARAMS = {
    #'tokenizer': tweet_tok.tokenize,
    'analyzer': 'word',
    'max_df': 0.5431858441280838,
    'use_idf': True,
    'smooth_idf': True,
    'sublinear_tf': True,
    'ngram_range': (1,4),
    'lowercase': True
   # 'stop_words': STOPWORDS

}

SVC_PARAMS  = {
    'fit_intercept': True,
    #'max_iter': 2900,
    'tol': 0.019114513977298204,
    'C': 0.7962740670237443,
    'class_weight': 'balanced',
    'penalty': 'l2',
    'multi_class': 'ovr'
    }

In [11]:

SVC_pipeline = Pipeline([
                    ('tfidf', TfidfVectorizer(**TF_PARAMS)),
                    ('clf', OneVsRestClassifier(CalibratedClassifierCV(LinearSVC(**SVC_PARAMS)))),
                     
                ])

SVC_pipeline.fit(train_data.text, train_data.label)
y_preds = SVC_pipeline.predict(test_data.text)
report = classification_report(test_data.label, y_preds)
print(str(np.mean(test_data.label == y_preds)))
print(report)



0.9094
              precision    recall  f1-score   support

           0       0.92      0.90      0.91      2495
           1       0.90      0.92      0.91      2505

    accuracy                           0.91      5000
   macro avg       0.91      0.91      0.91      5000
weighted avg       0.91      0.91      0.91      5000



### Define & Wrap predict_prob function

In [12]:
def predict_proba(inputs):

    return SVC_pipeline.predict_proba(inputs)

In [13]:
from checklist.pred_wrapper import PredictorWrapper
wrapped_pp = PredictorWrapper.wrap_softmax(predict_proba)

In [14]:
wrapped_pp(['This is not a good book', 'This is a good book', 'This book is okay', 'I do not like this book'])

(array([0, 1, 0, 0], dtype=int64),
 array([[0.99174398, 0.00825602],
        [0.01145994, 0.98854006],
        [0.99761106, 0.00238894],
        [0.90313039, 0.09686961]]))

### Import pre-defined Checklist Test Suite 

In [20]:
##Import checlist
import checklist
from checklist.test_suite import TestSuite
suite_path = 'sentiment_suite.pkl'
suite = TestSuite.from_file(suite_path)

In [21]:
suite.run(wrapped_pp, overwrite = True)

Running single positive words
Predicting 19 examples
Running single negative words
Predicting 23 examples
Running Sentiment-laden words in context
Predicting 4662 examples
Running simple negations: negative
Predicting 3402 examples
Running simple negations: not negative
Predicting 3654 examples


In [22]:
suite.visual_summary_table()

Please wait as we prepare the table data...


SuiteSummarizer(stats={'npassed': 0, 'nfailed': 0, 'nfiltered': 0}, test_infos=[{'name': 'single positive word…